In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('song_data-test.csv')
print(df.shape)
df.tail()

(391969, 21)


,title,artist,area,youtube_link,spotify_link,youtube_view_count,youtube_like_count,acousticness,danceability,duration_ms,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
391964,Come Back Home,Calum Scott,NaN,NaN,0cvfSKcm9VeduwyYPrxtLx,NaN,NaN,0.00678,0.601,190539.0,...,0.0,11.0,0.0991,-5.174,1.0,0.0323,131.049,4.0,0.2890,57.0
391965,Enough for You,Saint Claire,NaN,NaN,43MP9F7UzvfilSrw2SqZGJ,NaN,NaN,0.91800,0.387,194583.0,...,0.0,9.0,0.1030,-13.233,1.0,0.0437,94.039,4.0,0.3460,60.0
391966,Do It,Mike Stud,NaN,NaN,4TWlUuFk81NGUNKwndyS5Q,NaN,NaN,0.33000,0.717,139191.0,...,0.0,8.0,0.0997,-8.351,0.0,0.2060,156.977,4.0,0.5460,47.0
391967,No Smoke,D Savage,NaN,NaN,5iGBXzOoRo4sBTy8wdzMyK,NaN,NaN,0.00790,0.772,180013.0,...,0.0,4.0,0.1310,-9.670,0.0,0.1200,120.049,4.0,0.0755,50.0
391968,En Vida,Banda Los Sebastianes,NaN,NaN,7LNtyuekYHiZ99UxkrfCQR,NaN,NaN,0.54900,0.715,145827.0,...,0.0,3.0,0.1080,-3.244,1.0,0.0367,130.128,3.0,0.9760,55.0


In [4]:
df['artist']

0                      "#Botsu"
1         "El Cholo" Valderrama
2         "El Cholo" Valderrama
3         "El Cholo" Valderrama
4         "El Cholo" Valderrama
                  ...          
391964              Calum Scott
391965             Saint Claire
391966                Mike Stud
391967                 D Savage
391968    Banda Los Sebastianes
Name: artist, Length: 391969, dtype: object

In [5]:
import random
seed_value = 42
np.random.seed(seed_value)
dff = df.sample(n=10000,).reset_index()
#Set up hàng x cột ở đây, đang bị tràn bộ nhớ nên set 10000
print("Columns:", dff.columns)

Columns: Index(['index', 'title', 'artist', 'area', 'youtube_link', 'spotify_link',
       'youtube_view_count', 'youtube_like_count', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence', 'popularity'],
      dtype='object')


In [6]:
dff.drop(['duration_ms','youtube_link','spotify_link'],axis=1,inplace=True)

In [7]:
dfff = dff

In [8]:
dff = dff[['artist','youtube_view_count','acousticness','danceability','energy','instrumentalness','key','liveness','loudness','mode','speechiness','tempo','time_signature','valence','popularity']]

In [9]:
import ast

def fetch_list(strings):
    try:
        result = ast.literal_eval(strings)
        return result if isinstance(result, list) else [result]
    except (SyntaxError, ValueError):
        try:
            return [s.strip('"\'') for s in strings.split(',')]
        except Exception as e:
            print(f"Error processing expression in {strings}. Error: {e}")
            return []

# Print the 'artist' column before applying the fetch_list function
print("Before:")
print(dff['artist'])

# Apply the modified fetch_list function to the 'artist' column
dff['artist'] = dff['artist'].apply(fetch_list)

# Print the 'artist' column after applying the fetch_list function
print("\nAfter:")
print(dff['artist'])


Before:
0                      LANY
1             Peter Gregson
2                Mando Diao
3       The Andrews Sisters
4                   Sofiane
               ...         
9995             Los Llopis
9996    Penthouse Penthouse
9997           Jan Akkerman
9998              Stereolab
9999           Stevie Stone
Name: artist, Length: 10000, dtype: object

After:
0                      [LANY]
1             [Peter Gregson]
2                [Mando Diao]
3       [The Andrews Sisters]
4                   [Sofiane]
                ...          
9995             [Los Llopis]
9996    [Penthouse Penthouse]
9997           [Jan Akkerman]
9998              [Stereolab]
9999           [Stevie Stone]
Name: artist, Length: 10000, dtype: object


In [10]:
dff['artist'] = dff['artist'].apply(lambda x: " ".join(map(str, x)))

In [11]:
dff['artist'] = dff['artist'].apply(lambda x: x.lower())

In [12]:
dff['artist'].value_counts()

artist
johann sebastian bach      80
wolfgang amadeus mozart    37
armin van buuren           24
frank zappa                16
sweet                      14
                           ..
munya                       1
tasha                       1
ariana grande               1
phife dawg                  1
stevie stone                1
Name: count, Length: 4920, dtype: int64

In [13]:
dff['artist'].unique()

array(['lany', 'peter gregson', 'mando diao', ...,
       'strawberry switchblade', 'penthouse penthouse', 'stevie stone'],
      dtype=object)

In [14]:
dff['artist']=pd.factorize(dff['artist'])[0]

In [15]:
dff

,artist,youtube_view_count,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,0,NaN,0.322000,0.702,0.6230,0.0000,0.0,0.0858,-8.360,1.0,0.0417,104.974,4.0,0.503,75.0
1,1,NaN,0.000153,0.567,0.9620,0.0000,0.0,0.0598,-4.486,1.0,0.0661,115.018,4.0,0.576,NaN
2,2,71407.0,0.552000,0.571,0.3880,0.0000,4.0,0.1140,-11.671,1.0,0.0265,102.009,4.0,0.196,NaN
3,3,NaN,0.968000,0.430,0.0804,0.0000,0.0,0.0885,-13.574,1.0,0.0560,138.613,3.0,0.169,NaN
4,4,NaN,0.220000,0.522,0.8690,0.0000,1.0,0.1210,-3.959,1.0,0.3230,93.838,3.0,0.380,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,479,NaN,0.030500,0.659,0.5860,0.0966,2.0,0.0702,-12.124,0.0,0.0381,135.915,4.0,0.969,NaN
9996,4918,NaN,0.005270,0.737,0.5190,0.6540,0.0,0.1440,-7.849,0.0,0.0363,92.934,4.0,0.293,14.0
9997,2262,1411.0,0.156000,0.792,0.4210,0.0042,9.0,0.1120,-8.030,1.0,0.0703,89.994,4.0,0.542,NaN
9998,327,1032.0,0.433000,0.700,0.5110,0.0000,11.0,0.0562,-10.098,0.0,0.0405,87.372,4.0,0.685,NaN


In [16]:
dff['mode'] = dff['mode'].astype(bool)
dff=dff.fillna(0)
dff
# dff['explicit'] = dff['explicit'].astype(bool)

,artist,youtube_view_count,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,0,0.0,0.322000,0.702,0.6230,0.0000,0.0,0.0858,-8.360,True,0.0417,104.974,4.0,0.503,75.0
1,1,0.0,0.000153,0.567,0.9620,0.0000,0.0,0.0598,-4.486,True,0.0661,115.018,4.0,0.576,0.0
2,2,71407.0,0.552000,0.571,0.3880,0.0000,4.0,0.1140,-11.671,True,0.0265,102.009,4.0,0.196,0.0
3,3,0.0,0.968000,0.430,0.0804,0.0000,0.0,0.0885,-13.574,True,0.0560,138.613,3.0,0.169,0.0
4,4,0.0,0.220000,0.522,0.8690,0.0000,1.0,0.1210,-3.959,True,0.3230,93.838,3.0,0.380,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,479,0.0,0.030500,0.659,0.5860,0.0966,2.0,0.0702,-12.124,False,0.0381,135.915,4.0,0.969,0.0
9996,4918,0.0,0.005270,0.737,0.5190,0.6540,0.0,0.1440,-7.849,False,0.0363,92.934,4.0,0.293,14.0
9997,2262,1411.0,0.156000,0.792,0.4210,0.0042,9.0,0.1120,-8.030,True,0.0703,89.994,4.0,0.542,0.0
9998,327,1032.0,0.433000,0.700,0.5110,0.0000,11.0,0.0562,-10.098,False,0.0405,87.372,4.0,0.685,0.0


In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
vectors = dff.to_numpy()

In [19]:
vectors

array([[0, 0.0, 0.322, ..., 4.0, 0.503, 75.0],
       [1, 0.0, 0.000153, ..., 4.0, 0.576, 0.0],
       [2, 71407.0, 0.552, ..., 4.0, 0.196, 0.0],
       ...,
       [2262, 1411.0, 0.156, ..., 4.0, 0.542, 0.0],
       [327, 1032.0, 0.433, ..., 4.0, 0.685, 0.0],
       [4919, 1585.0, 0.0089, ..., 4.0, 0.48, 28.0]], dtype=object)

In [20]:
similarity = cosine_similarity(vectors)

In [21]:
similarity.shape

(10000, 10000)

In [22]:
sorted(similarity[0],reverse=True)

[1.0,
 0.9842435996311671,
 0.9808035286802279,
 0.9745724001269872,
 0.9591263789834923,
 0.9456783965964656,
 0.9353481270735954,
 0.9343319995233895,
 0.933578063396312,
 0.9335288192872414,
 0.9325504416107199,
 0.9312923225824964,
 0.931197521535504,
 0.9292859868608984,
 0.9275912489216759,
 0.9238994662720571,
 0.9191184109313546,
 0.9160530395491178,
 0.9146410752958714,
 0.9118118217327279,
 0.9117900839392291,
 0.9116325838557819,
 0.9070338375445007,
 0.9049512983414338,
 0.8999009346844133,
 0.8991689137036564,
 0.8967682500269744,
 0.895466415684704,
 0.8937234467695099,
 0.8915651365762081,
 0.8877707089258686,
 0.886426671765425,
 0.8859212940092944,
 0.88514158535357,
 0.8830709001517459,
 0.8814532615260582,
 0.8811251071594871,
 0.8786199214364262,
 0.8772601509828867,
 0.8766894963458314,
 0.8763383262839961,
 0.8732313902960867,
 0.872707329272365,
 0.8683343560148334,
 0.86710342878569,
 0.8670551449582007,
 0.8661789229780524,
 0.8659201485003748,
 0.8653654140213

In [23]:
list(enumerate(similarity[0]))

[(0, 1.0),
 (1, 0.8140490905144484),
 (2, 0.001171805387146007),
 (3, 0.8142955026868589),
 (4, 0.9808035286802279),
 (5, 0.8763383262839961),
 (6, 0.003783553374651819),
 (7, 8.452027827554876e-06),
 (8, 0.8059924520749027),
 (9, 0.8131659182000956),
 (10, 3.320744766053422e-05),
 (11, 0.805141868473028),
 (12, 0.8121091996281863),
 (13, 2.6911864927902907e-05),
 (14, 0.8098789395950606),
 (15, 0.0021893854537882516),
 (16, 7.220159521996373e-05),
 (17, 0.8210553622939579),
 (18, 0.0002653184351966247),
 (19, 0.9335288192872414),
 (20, 0.04397664543734736),
 (21, 7.758800365932742e-05),
 (22, 0.8011780901550865),
 (23, 0.895466415684704),
 (24, 0.9049512983414338),
 (25, 0.0006095328225413198),
 (26, 5.042512548334585e-06),
 (27, 0.7790445195257212),
 (28, 0.7961396555590463),
 (29, 0.00010859835703358966),
 (30, 0.0013703827568394668),
 (31, 0.11026963293770445),
 (32, 0.8915651365762081),
 (33, 0.0001282672421351904),
 (34, 0.0012614969243069435),
 (35, 0.9325504416107199),
 (36, 0.

In [24]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:11]

[(1801, 0.9842435996311671),
 (4, 0.9808035286802279),
 (275, 0.9745724001269872),
 (4948, 0.9591263789834923),
 (2704, 0.9456783965964656),
 (6405, 0.9353481270735954),
 (4447, 0.9343319995233895),
 (2332, 0.933578063396312),
 (19, 0.9335288192872414),
 (35, 0.9325504416107199)]

In [33]:
#dfff[dfff["name"]=='Sister'].index
def recommend(song):
    song_index = dfff[dfff["title"]==song].index[0]
    distance = similarity[song_index]
    songs_list = sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:11]
    for i in songs_list:
        print(dfff.iloc[i[0]]['title'])

In [34]:
dfff.tail()

,index,title,artist,area,youtube_view_count,youtube_like_count,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
9995,72345,Castigo y cariñitos,Los Llopis,Latin,NaN,NaN,0.03050,0.659,0.586,0.0966,2.0,0.0702,-12.124,0.0,0.0381,135.915,4.0,0.969,NaN
9996,332508,Space Vacation,Penthouse Penthouse,NaN,NaN,NaN,0.00527,0.737,0.519,0.6540,0.0,0.1440,-7.849,0.0,0.0363,92.934,4.0,0.293,14.0
9997,32450,Quiet Storm,Jan Akkerman,USUK,1411.0,18.0,0.15600,0.792,0.421,0.0042,9.0,0.1120,-8.030,1.0,0.0703,89.994,4.0,0.542,NaN
9998,127190,Contronatura,Stereolab,USUK,1032.0,22.0,0.43300,0.700,0.511,0.0000,11.0,0.0562,-10.098,0.0,0.0405,87.372,4.0,0.685,NaN
9999,271796,Envy,Stevie Stone,KR,1585.0,18.0,0.00890,0.679,0.726,0.0000,10.0,0.0838,-6.461,0.0,0.1350,157.900,4.0,0.480,28.0


In [35]:
recommend('Space Vacation')

Breaking News (feat. Tokyo Jetz)
Whoop Ass
vines
Hallelujah
Intro
Am I the Only One?
My City
Obimbra (Say No to Rape)
Lost Angeles
Roaming in Paris
